In [1]:
import dask
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from scipy import sparse
import psutil
NUM_WORKER = psutil.cpu_count(logical = False)

In [2]:
df = dd.read_csv('../data/tests/interim/features/*.csv')

In [3]:
df.head()

,api,invocation,block,app,malware
0,La/a;-><init>,invoke-direct,static constructor <clinit>()Va.smali,%D8%AA%D9%90%D9%82%D9%86%D9%8A,0
1,Ljava/lang/Runtime;->getRuntime,invoke-static,static constructor <clinit>()Va.smali,%D8%AA%D9%90%D9%82%D9%86%D9%8A,0
2,Ljava/lang/Runtime;->availableProcessors,invoke-virtual,static constructor <clinit>()Va.smali,%D8%AA%D9%90%D9%82%D9%86%D9%8A,0
3,Ljava/lang/Object;-><init>,invoke-direct,private constructor <init>()Va.smali,%D8%AA%D9%90%D9%82%D9%86%D9%8A,0
4,La/b;-><init>,invoke-direct,private constructor <init>()Va.smali,%D8%AA%D9%90%D9%82%D9%86%D9%8A,0
